In [1]:
import numpy as np

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Activation, Dense, Conv2D, MaxPooling2D, ZeroPadding2D, Flatten ,Dropout
from keras.models import Sequential, Model
from keras import applications
from keras.optimizers import SGD
from keras.callbacks import TensorBoard

from sklearn.metrics import classification_report, confusion_matrix

Using TensorFlow backend.


In [2]:
img_width = 224
img_height = 224
split = 0.2
train_samples = 5000 *(1-split)
validation_samples =  5000*split
epochs = 50
batch_size = 16
num_classes = 10

dataset = './dataset/training_set'
datatest = './dataset/test_set'

In [8]:
#Image Augementation train
image_generator = ImageDataGenerator(
    shear_range=0.05,
    rescale=1. / 255,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5],
    validation_split=split,
    )

train_generator = image_generator.flow_from_directory(
    dataset,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    subset="training",
    class_mode='categorical')

validation_generator = image_generator.flow_from_directory(
    dataset,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset="validation")

Found 360 images belonging to 10 classes.
Found 84 images belonging to 10 classes.


In [4]:
base_model = applications.mobilenet_v2.MobileNetV2(weights = "imagenet",\
                                                   include_top=False, input_shape\
                                                   = (img_width, img_height, 3))

In [5]:
top_model = Flatten(name="flatten")(base_model.output)
top_model = Dense(512,activation="relu")(top_model)
top_model = Dropout(0.5)(top_model)
top_model = Dense(num_classes,activation="softmax")(top_model)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
for layer in base_model.layers[:19]:
    layer.trainable = False
    
final_model = Model(inputs=base_model.input,outputs=top_model)    

### MobileNetV2 training :80% validation 20%

In [9]:
print("[INFO] compiling model...")
opt = SGD(lr=1e-4, momentum=0.9)
final_model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy","binary_crossentropy"])


#Model Fitting
callbacks = TensorBoard(log_dir='.\\Graph\\MobileNetV2')
H = final_model.fit_generator(
	train_generator,
	steps_per_epoch=train_samples // batch_size,
	validation_data=validation_generator,
	validation_steps=validation_samples // batch_size,
	epochs=epochs,
    callbacks=[callbacks])

print("[INFO] serializing network...")
final_model.save("MobileNetV2.h5")

[INFO] compiling model...
Epoch 1/50
250/250 [==============================] - 99s 396ms/step - loss: 0.0980 - acc: 0.9632 - binary_crossentropy: 0.0980 - val_loss: 0.1366 - val_acc: 0.9483 - val_binary_crossentropy: 0.1366
Epoch 2/50
250/250 [==============================] - 94s 376ms/step - loss: 0.0895 - acc: 0.9669 - binary_crossentropy: 0.0895 - val_loss: 0.1403 - val_acc: 0.9482 - val_binary_crossentropy: 0.1403
Epoch 3/50
250/250 [==============================] - 92s 367ms/step - loss: 0.0766 - acc: 0.9711 - binary_crossentropy: 0.0766 - val_loss: 0.1356 - val_acc: 0.9487 - val_binary_crossentropy: 0.1356
Epoch 4/50
250/250 [==============================] - 92s 370ms/step - loss: 0.0692 - acc: 0.9749 - binary_crossentropy: 0.0692 - val_loss: 0.1392 - val_acc: 0.9485 - val_binary_crossentropy: 0.1392
Epoch 5/50
250/250 [==============================] - 92s 369ms/step - loss: 0.0687 - acc: 0.9756 - binary_crossentropy: 0.0687 - val_loss: 0.1292 - val_acc: 0.9533 - val_binary_

Epoch 42/50
250/250 [==============================] - 92s 370ms/step - loss: 0.0142 - acc: 0.9948 - binary_crossentropy: 0.0142 - val_loss: 0.1242 - val_acc: 0.9585 - val_binary_crossentropy: 0.1242
Epoch 43/50
250/250 [==============================] - 93s 371ms/step - loss: 0.0148 - acc: 0.9944 - binary_crossentropy: 0.0148 - val_loss: 0.1294 - val_acc: 0.9594 - val_binary_crossentropy: 0.1294
Epoch 44/50
250/250 [==============================] - 95s 378ms/step - loss: 0.0145 - acc: 0.9945 - binary_crossentropy: 0.0145 - val_loss: 0.1342 - val_acc: 0.9553 - val_binary_crossentropy: 0.1342
Epoch 45/50
250/250 [==============================] - 92s 369ms/step - loss: 0.0176 - acc: 0.9936 - binary_crossentropy: 0.0176 - val_loss: 0.1325 - val_acc: 0.9594 - val_binary_crossentropy: 0.1325
Epoch 46/50
250/250 [==============================] - 92s 367ms/step - loss: 0.0160 - acc: 0.9937 - binary_crossentropy: 0.0160 - val_loss: 0.1234 - val_acc: 0.9595 - val_binary_crossentropy: 0.1234


In [20]:
image_test_generator = ImageDataGenerator(rescale=1/255)

testing_generator = image_test_generator.flow_from_directory(
    datatest,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
    )

Found 134 images belonging to 10 classes.


In [21]:
final_model.evaluate_generator(testing_generator,steps= 134 // batch_size+1,workers=12)


[0.12629613080131474, 0.9589552167636245, 0.12629613080131474]

In [22]:
testing_generator.reset()
Y_pred = final_model.predict_generator(testing_generator, 134 // batch_size+1,workers=12)


In [23]:
y_pred = np.argmax(Y_pred, axis=-1)
print('Confusion Matrix')
print(confusion_matrix(testing_generator.classes, y_pred))

Confusion Matrix
[[12  0  0  0  0  1  1  0  1  0]
 [ 0 13  0  0  1  0  0  0  1  0]
 [ 0  0  2  0  0  2  0  0  0  0]
 [ 0  0  0 11  2  2  1  0  0  0]
 [ 0  0  0  0 10  1  1  0  0  0]
 [ 0  0  0  0  0 17  1  0  0  0]
 [ 0  0  0  0  0  0 14  0  0  0]
 [ 0  0  0  0  1  1  0  9  1  0]
 [ 1  0  0  0  0  1  1  1 12  0]
 [ 0  0  0  0  0  0  2  0  1  9]]


In [24]:
batik_index = ['Batik Kawung','Batik Megamendung','Batik Nitik','Batik Parang',
               'Batik Sido Luhur','Batik Truntum','Batik Udan Liris','Batik Gedok','Batik Ceplok','Batik Tambal']
print('Classification Report')
print(classification_report(testing_generator.classes, y_pred, target_names=batik_index))

Classification Report
                   precision    recall  f1-score   support

     Batik Kawung       0.92      0.80      0.86        15
Batik Megamendung       1.00      0.87      0.93        15
      Batik Nitik       1.00      0.50      0.67         4
     Batik Parang       1.00      0.69      0.81        16
 Batik Sido Luhur       0.71      0.83      0.77        12
    Batik Truntum       0.68      0.94      0.79        18
 Batik Udan Liris       0.67      1.00      0.80        14
      Batik Gedok       0.90      0.75      0.82        12
     Batik Ceplok       0.75      0.75      0.75        16
     Batik Tambal       1.00      0.75      0.86        12

         accuracy                           0.81       134
        macro avg       0.86      0.79      0.81       134
     weighted avg       0.85      0.81      0.82       134



### JIka model sudah disimpan

In [3]:
from keras.models import load_model

In [4]:
loaded_model = load_model("MobileNetV2.h5")












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [7]:
testing_generator.reset()
loaded_model.evaluate_generator(testing_generator,steps= 134 // batch_size+1,workers=12)

[0.6565202625829782, 0.8470149422759441, 0.6565202625829782]

In [9]:
testing_generator.reset()
Y_pred = loaded_model.predict_generator(testing_generator, 134 // batch_size+1,workers=12)

In [11]:
y_pred = np.argmax(Y_pred, axis=-1)
print('Confusion Matrix')
print(confusion_matrix(testing_generator.classes, y_pred))

Confusion Matrix
[[ 4  0  0  0  0 11  0  0  0  0]
 [ 1  5  0  0  0  9  0  0  0  0]
 [ 1  0  0  0  0  3  0  0  0  0]
 [ 1  2  0  2  1 10  0  0  0  0]
 [ 1  0  0  0  1 10  0  0  0  0]
 [ 1  0  0  0  0 17  0  0  0  0]
 [ 1  0  0  2  0 11  0  0  0  0]
 [ 2  0  0  0  0  9  0  1  0  0]
 [ 0  0  0  0  0 16  0  0  0  0]
 [ 0  0  0  0  0 11  1  0  0  0]]


In [12]:
batik_index = ['Batik Kawung','Batik Megamendung','Batik Nitik','Batik Parang','Batik Sido Luhur','Batik Truntum','Batik Udan Liris','Batik Gedok','Batik Ceplok','Batik Tambal']
print('Classification Report')
print(classification_report(testing_generator.classes, y_pred, target_names=batik_index))

Classification Report
                   precision    recall  f1-score   support

     Batik Kawung       0.33      0.27      0.30        15
Batik Megamendung       0.71      0.33      0.45        15
      Batik Nitik       0.00      0.00      0.00         4
     Batik Parang       0.50      0.12      0.20        16
 Batik Sido Luhur       0.50      0.08      0.14        12
    Batik Truntum       0.16      0.94      0.27        18
 Batik Udan Liris       0.00      0.00      0.00        14
      Batik Gedok       1.00      0.08      0.15        12
     Batik Ceplok       0.00      0.00      0.00        16
     Batik Tambal       0.00      0.00      0.00        12

         accuracy                           0.22       134
        macro avg       0.32      0.18      0.15       134
     weighted avg       0.33      0.22      0.17       134



D:\Anaconda\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
